# Entrenamiento y predicción de modelos

In [2]:
import nb_black
import pandas as pd
import numpy as np
import openpyxl
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar datos
ventas_h = pd.read_excel('../data/Ventas_históricas.xlsx',parse_dates=['Fecha'])
ventas_h

,Fecha,Producto A,Producto B,Producto C,Producto D
0,2018-01-01,100,120,90,80
1,2018-02-01,107,123,92,86
2,2018-03-01,111,127,97,88
3,2018-04-01,118,134,101,93
4,2018-05-01,121,139,105,97
...,...,...,...,...,...
67,2023-08-01,441,506,391,359
68,2023-09-01,446,512,396,363
69,2023-10-01,452,518,401,368
70,2023-11-01,457,524,406,372


### Preprocesamiento de datos

In [3]:

def preprocess_data(data_ventas):
    data_ventas['mes'] = data_ventas['Fecha'].dt.month
    data_ventas['año'] = data_ventas['Fecha'].dt.year
    data_ventas = data_ventas.groupby(['año', 'mes']).agg({'Producto A':'sum' , 'Producto B': 'sum', 'Producto C': 'sum', 'Producto D':'sum'}).reset_index()
    return data_ventas

ventas_agrupadas = preprocess_data(ventas_h)
ventas_agrupadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   año         72 non-null     int32
 1   mes         72 non-null     int32
 2   Producto A  72 non-null     int64
 3   Producto B  72 non-null     int64
 4   Producto C  72 non-null     int64
 5   Producto D  72 non-null     int64
dtypes: int32(2), int64(4)
memory usage: 2.9 KB


In [4]:
ventas_agrupadas

,año,mes,Producto A,Producto B,Producto C,Producto D
0,2018,1,100,120,90,80
1,2018,2,107,123,92,86
2,2018,3,111,127,97,88
3,2018,4,118,134,101,93
4,2018,5,121,139,105,97
...,...,...,...,...,...,...
67,2023,8,441,506,391,359
68,2023,9,446,512,396,363
69,2023,10,452,518,401,368
70,2023,11,457,524,406,372


### Evaluación del modelo de regresión lineal en cada producto

In [5]:
def evaluacion_modelo_productos(data):
    productos = ['Producto A', 'Producto B', 'Producto C', 'Producto D']
    precision_modelos = {}
    
    for producto in productos:
        # Filtrar los datos para el producto actual
        product_data = data[['año', 'mes', producto]].dropna()
        # Dividir en X (variables independientes) e y (variable dependiente)
        x = ventas_agrupadas[['año','mes']]
        y = product_data[producto]

        x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

        # Crear y entrenar el model
        md_regresion_lineal = LinearRegression()
        md_regresion_lineal.fit(x_train, y_train)

        # Predicción
        Y_pred = md_regresion_lineal.predict(x_test)
        
        # Calcular métricas de evaluación
        mse = mean_squared_error(y_test, Y_pred)
        mae = mean_absolute_error(y_test, Y_pred)
        r2 = r2_score(y_test, Y_pred)
                
        # Almacenar los resultados
        if producto not in precision_modelos:
            precision_modelos[producto] = {}
        precision_modelos[producto] = {'MSE': mse, 'MAE': mae, 'R2': r2}  

        #Guardar modelos
        if producto == 'Producto A':
            joblib.dump(md_regresion_lineal, "../Modelos_creados/MODEL_RL_producto_A.pkl")
        elif producto == 'Producto B':
            joblib.dump(md_regresion_lineal, "../Modelos_creados/MODEL_RL_producto_B.pkl")
        elif producto == 'Producto C':
            joblib.dump(md_regresion_lineal, "../Modelos_creados/MODEL_RL_producto_C.pkl")
        else:
            joblib.dump(md_regresion_lineal, "../Modelos_creados/MODEL_RL_producto_D.pkl")
        
    return precision_modelos

### Resultados de la métricas evaluadas

In [6]:
evaluacion_modelos = evaluacion_modelo_productos(ventas_agrupadas) 
evaluacion_modelos = pd.DataFrame(evaluacion_modelos)
evaluacion_modelos

,Producto A,Producto B,Producto C,Producto D
MSE,10.709271,13.126581,21.945663,12.616778
MAE,2.500852,2.755546,4.019196,3.196922
R2,0.998928,0.999004,0.997174,0.998072


### Cargar modelo

In [7]:
#Funciones para cargar modelos

def load_modelo_RL_PA():
    return joblib.load("../Modelos_creados/MODEL_RL_producto_A.pkl")

def load_modelo_RL_PB():
    return joblib.load("../Modelos_creados/MODEL_RL_producto_B.pkl")

def load_modelo_RL_PC():
    return joblib.load("../Modelos_creados/MODEL_RL_producto_C.pkl")

def load_modelo_RL_PD():
    return joblib.load("../Modelos_creados/MODEL_RL_producto_D.pkl")


### Predicción de demanda para el año 2025 por cada mes

In [14]:
#Data para predicción
año = 2024
mes = [1,2,3,4,5,6,7,8,9,10,11,12]
input_data = pd.DataFrame({'año': año, 'mes': mes})
input_data = input_data.reset_index(drop=True)  
#input_data.index = input_data.index + 1 
input_data


,año,mes
0,2024,1
1,2024,2
2,2024,3
3,2024,4
4,2024,5
5,2024,6
6,2024,7
7,2024,8
8,2024,9
9,2024,10


In [15]:

modelo_PA = load_modelo_RL_PA()
prediction = modelo_PA.predict(input_data)
prediction.round()

array([464., 469., 475., 480., 485., 490., 495., 500., 505., 510., 515.,
       520.])

In [16]:
# Cargar los modelos
modelo_PA = load_modelo_RL_PA()
modelo_PB = load_modelo_RL_PB()
modelo_PC = load_modelo_RL_PC()
modelo_PD = load_modelo_RL_PD()
# Realizar la predicción y convertir las predicciones en un DataFrame
predictionA = modelo_PA.predict(input_data)
predictionA = np.ceil(predictionA)
predictionsA_df = pd.DataFrame(predictionA , columns=['Predicciones_PA'])

predictionB = modelo_PB.predict(input_data)
predictionB = np.ceil(predictionB)
predictionsB_df = pd.DataFrame(predictionB , columns=['Predicciones_PB'])

predictionC = modelo_PC.predict(input_data)
predictionC = np.ceil(predictionC)
predictionsC_df = pd.DataFrame(predictionC , columns=['Predicciones_PC'])

predictionD = modelo_PD.predict(input_data)
predictionD = np.ceil(predictionD)
predictionsD_df = pd.DataFrame(predictionD , columns=['Predicciones_PD'])

# Unir las predicciones con los datos de entrada
data_output = pd.concat([input_data, predictionsA_df, predictionsB_df, predictionsC_df, predictionsD_df], axis=1)

#Guardamos la tabla de predicciones
data_output.to_csv('../data/data_predicción.csv')

# Devolver el resultado para que Power BI lo utilice
data_output

,año,mes,Predicciones_PA,Predicciones_PB,Predicciones_PC,Predicciones_PD
0,2024,1,465.0,534.0,409.0,376.0
1,2024,2,470.0,540.0,414.0,380.0
2,2024,3,475.0,546.0,419.0,384.0
3,2024,4,480.0,552.0,423.0,389.0
4,2024,5,485.0,557.0,428.0,393.0
5,2024,6,490.0,563.0,433.0,397.0
6,2024,7,495.0,569.0,437.0,402.0
7,2024,8,501.0,575.0,442.0,406.0
8,2024,9,506.0,581.0,447.0,410.0
9,2024,10,511.0,587.0,451.0,414.0
